### Intermediate Deliverable 2

- $I=\{DSO545-16278,DSO528-16274,....\}$ Course+Section
- $Im=\{DSO433-16227,DSO516-16305,....\}$ Course+Section that takes place in multiple days of the week
- $J=\{JKP102,JKP104,...\}$ Room
- $K=\{M, T, W, H, F, S, U\}$ Day of the week from Monday to Sunday
- $Ki=$ days of the week that course+section i takes place
- $L=\{0,1,...,14\}$ index of the time slot of each day from 10:00 to 17:00; each time slot is 30 mins.
- Let decision variables be $x_{ijkl}$ to denote whether course+section $i \in I$ assigned to room $j \in J$ on day $k \in K$ at time slot $l \in L$.
- Let $S_{i}$ denote the number of registered students for course+section $i \in I$.
- Let $C_{j}$ denote the capacity of room $j \in J$.
- Let $u_{i}$ denote the total number of slots needed for course+section $i \in I$ for one day. For example, for a 3-hour class, a total of 6 slots is needed; for a 1.5-hour class, a total of 3 slots is needed for a single day.
- Let auxiliary decision variable $U_{ijk}$ corresponding to an upper bound of $x_{ijkl}$'s that belong to the same course+section $i$, room $j$ and day $k$. For example, for course+section $i$, $U_{ijk}$ would be 1 if any of the slots at room $j$ on day $k$ is filled.
- Let $Y_{ik}=\{0,1\}$ denote whether course+section $i$ takes place on day $k$.

$$\begin{aligned}
\text{maximize} && \sum_{i \in I}\sum_{j \in J}\sum_{k \in K}\sum_{l \in L} x_{ijkl}* \frac{S_{i}}{C_{j}}\\\
\text{classroom capacity constraint}&& \ x_{ijkl}*S_{i} \le 0.9*C_{j} & \qquad \text{for all $j \in J$}&\\
\text{day of week constraint}&& \sum_{j \in J}\sum_{l \in L} x_{ijkl} = u_{i}*Y_{ik} & \qquad \text{for all $i \in I$, $k \in K$}&\\
\text{one course constraint}&& \sum_{i \in I} x_{ijkl} \le 1 & \qquad \text{for all $j \in J$, $k \in K$, $l \in L$}&\\
\text{unique room + time slots constraint}&& \sum_{j \in J} x_{ijkl} \le 1 & \qquad \text{for all $i \in I$, $k \in K$, $l \in L$}&\\
\text{}&& \ x_{ijkl} \le U_{ijk} & \qquad \text{for all $i \in I$, $j \in J$, $k \in K$, $l \in L$}&\\
\text{}&& \sum_{l \in L} x_{ijkl} = u_{i}*U_{ijk} & \qquad \text{for all $i \in I$, $k \in K$, $j \in J$}&\\
\text{consecutive slots constraint}&& \ x_{ijkl} + x_{ijkl'} \le 1 & \qquad \text{for all $i \in I$, $j \in J$, $k \in K$, $l \in L$ except for the last $u_{i}$ $l$s, $l' \ge l+u_{i}-1$} &\\
\text{same slots for multiple-day class constraint}&& \ x_{ijkl} = x_{ij(k+1)l} & \qquad \text{for all $i \in Im$, $j \in J$, $k \in Ki$ except the last k, $l \in L$}&\\
\text{instructor constraint}&& \  & \qquad \text{for all $i \in Im$, $j \in J$, $k \in Ki$ except the last k, $l \in L$}&\\
\end{aligned}$$

In [29]:
import gurobipy as grb
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math

file_name = 'small_data'
data = pd.ExcelFile('{}.xlsx'.format(file_name))

courses = pd.read_excel('{}.xlsx'.format(file_name), sheet_name='Course_Enrollment', parse_dates=False)#data.parse(0, parse_dates=True) #Courses
courses = courses.set_index(['Course', 'Section'])

rooms = data.parse(1) #Rooms
rooms = rooms.set_index('Room')

#students = data.parse(2) #Students

#AM8:00~PM10:00 total 14 hours, 28 time slots 
#For deliverable 2, we will only use prime time AM10:00~PM5:00 14 slots
slots_per_day = 14 
#num_days = 7

I = courses.index #Course name and section (tuple)
J = rooms.index #Room ID
K = ['M', 'T', 'W', 'H', 'F', 'S', 'U']
L = range(0,slots_per_day) #Time slot ID

Ki = courses.loc[:,'First Days']

s = courses.loc[:,'Reg Count'] #Registration Count
c = rooms.Size #Room Capacity

#Convert time difference to 30 mins unit slot: 3 h = 6 slots, 1 h 20 m = 3 slots
u2 = pd.to_datetime(courses.loc[:,'First End Time'].astype(str))#.apply(datetime)
u1 = pd.to_datetime(courses.loc[:,'First Begin Time'].astype(str))
u = ((u2-u1).apply(timedelta.total_seconds)/1800).apply(math.ceil).astype(int)

mod=grb.Model()

#Variable Definition
x={} #binary variable whether the course is assigned to each room at each time slot of each day
U={} #Maximum value of all slots for that day for each course and each room
for i in I:
    for j in J:
        for k in K:
            U[i,j,k]=mod.addVar(vtype=grb.GRB.BINARY,name='U[{0},{1},{2}]'.format(i,j,k))
            for l in L:
                x[i,j,k,l]=mod.addVar(vtype=grb.GRB.BINARY,name='x[{0},{1},{2},{3}]'.format(i,j,k,l))

#Classroom capacity constraint
for i in I:
    for j in J:
        for k in Ki[i]:
            for l in L:
                mod.addConstr(x[i,j,k,l]*s[i] <= 0.9*c[j])
                
#Day of week constraint
for i in I:
    for k in K:
        if k not in Ki[i]:
            mod.addConstr(sum(x[i,j,k,l] for j in J for l in L)==0)
        else:
            mod.addConstr(sum(x[i,j,k,l] for j in J for l in L)==u[i])

#One course constraint
for j in J:
    for k in K:
        for l in L:
            mod.addConstr(sum(x[i,j,k,l] for i in I) <= 1)

#Unique room constraint
for i in I:
    for k in K:
        for l in L:
            mod.addConstr(sum(x[i,j,k,l] for j in J) <= 1)
            
for i in I:
    for j in J:
        for k in K:
            for l in L:
                mod.addConstr(x[i,j,k,l] <= U[i,j,k])
            mod.addConstr(sum(x[i,j,k,l] for l in L) == u[i]*U[i,j,k])
            
#Consecutive constraint
for i in I:
    for j in J:
        for k in Ki[i]:
            for l in L:
                if l + u[i] < len(L) - 1:
                    for m in range(l + u[i], len(L)):
                        mod.addConstr(x[i,j,k,l] + x[i,j,k,m] <= 1)

#Same slot of day constraint for multiple days
for i in I:
    if len(Ki[i]) > 1:
        for j in J:
            for index, k in enumerate(Ki[i]):
                if index < len(Ki[i])-1:
                    for l in L:
                        mod.addConstr(x[i,j,Ki[i][index],l]==x[i,j,Ki[i][index+1],l])


#Objective function
mod.setObjective(sum(x[i,j,k,l]*s[i]/c[j] for i in I for j in J for k in K for l in L) , sense=grb.GRB.MAXIMIZE)

#Optimize
mod.optimize()
print('Optimal Objective:', mod.ObjVal)

#Save output
outTable1={}
time_slot = 0
for k in K:
    for l in L:
        data = []
        for i in I:    
            Room_id = ''
            for j in J:
                if x[i,j,k,l].x == 1 :
                    Room_id = j
                    break
            data.append(Room_id)
            
        outTable1[time_slot] = data
        time_slot += 1
    

outDf1 = pd.DataFrame(outTable1, index=I)

outTable2={}
time_slot = 0
for k in K:
    for l in L:
        data = []
        for j in J:    
            course_id = ''
            for i in I:
                if x[i,j,k,l].x == 1 :
                    course_id = "{}-{}:{}".format(i[0],i[1],s[i])
                    break
            data.append(course_id)
            
        outTable2[time_slot] = data
        time_slot += 1

outDf2 = pd.DataFrame(outTable2, index=[J,c])

writer = pd.ExcelWriter('output_of_{}.xlsx'.format(file_name))
outDf1.to_excel(writer,'Course')
outDf2.to_excel(writer,'Room')
writer.save()

Optimize a model with 74815 rows, 34650 columns and 260930 nonzeros
Variable types: 0 continuous, 34650 integer (34650 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [7e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Presolve removed 70548 rows and 32718 columns
Presolve time: 0.32s
Presolved: 4267 rows, 1932 columns, 22869 nonzeros
Variable types: 0 continuous, 1932 integer (1932 binary)
Found heuristic solution: objective 134.5046653

Root relaxation: objective 1.492118e+02, 1066 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  149.21179    0   62  134.50467  149.21179  10.9%     -    0s
H    0     0                     148.4405513  149.21179  0.52%     -    0s
     0     0  149.18238    0   37  148.44055  149.18238  0.50%     -    0s
     0     0  149.18238    0   68  148.44

In [23]:
print(rooms)

         Room  Size
0    ACC 306B    16
1      ACC201    48
2      ACC205    36
3      ACC236    39
4      ACC303    46
5     ACC306B    16
6      ACC310    54
7      ACC312    20
8      BRI202    42
9     BRI202A    34
10       BRI5    42
11       BRI8    36
12    HOH EDI   269
13       HOH1    73
14       HOH2    73
15     HOH506    16
16     HOH706    16
17  JFF LL101    48
18  JFF LL102    48
19  JFF LL103    48
20  JFF LL105   149
21  JFF LL125   101
22     JFF233    60
23     JFF236    60
24     JFF239    48
25     JFF240    48
26     JFF241    48
27     JFF312    20
28     JFF313    20
29     JFF316    48
30     JFF322    48
31     JFF327    36
32     JFF328    36
33     JFF331    36
34     JFF414    60
35     JFF416    48
36     JFF417    36
37     JKP102    52
38     JKP104    56
39     JKP110    77
40     JKP112    77
41     JKP202    54
42     JKP204    54
43     JKP210    78
44     JKP212    78
